## Analisis de Datos

In [67]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [68]:
dataset=pd.read_csv('../python-ml-course/datasets/wine/winequality-red.csv',sep=';')
dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [69]:
dataset.shape

(1599, 12)

### Normalizar dataset

In [70]:
data=(dataset-dataset.min())/(dataset.max()-dataset.min())
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.4
1,0.283186,0.520548,0.00,0.116438,0.143573,0.338028,0.215548,0.494126,0.362205,0.209581,0.215385,0.4
2,0.283186,0.438356,0.04,0.095890,0.133556,0.197183,0.169611,0.508811,0.409449,0.191617,0.215385,0.4
3,0.584071,0.109589,0.56,0.068493,0.105175,0.225352,0.190813,0.582232,0.330709,0.149701,0.215385,0.6
4,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.4


## Obtener numero optimo de clusters

### Metodo  Propagacion de la Afinidad

In [71]:
from sklearn.cluster import AffinityPropagation

In [72]:
af=AffinityPropagation(preference=-50).fit(data)

cluster_center_ids = af.cluster_centers_indices_

n_clust = len(cluster_center_ids)
n_clust

3

### Metodo con Scipy

In [73]:
from scipy.cluster.hierarchy import linkage,dendrogram
from scipy.cluster.hierarchy import fcluster

In [74]:
z=linkage(data,'ward')
z2=z[:,2] #se obtienen todas las distancias de los clusters calculados

def max_diff(dist_clusters):
    diff_clusters=np.diff(dist_clusters)
    max=0
    for index_i,i in enumerate(diff_clusters):
        if(i>max):
            max=i
            idx=index_i
        for index_j,j in enumerate(diff_clusters):
            if(j>max):
                max=j
                idx=index_j          
    return (dist_clusters[idx])

max_d=round(max_diff(z2),2)
max_d
clusters=fcluster(z,max_d,criterion='distance')
print('Numero de clusters optimos: '+ str(clusters.max()))

Numero de clusters optimos: 3


### Metodo de la silueta

In [75]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples,silhouette_score

In [76]:
max_clusters=len(data)
clusters=range(2,max_clusters) #clusters a partir de 2
resultado={}

In [77]:
for k in clusters:
    kmean_model=KMeans(n_clusters=k).fit(data)
    labels=kmean_model.labels_
    
    siluetas_temp=[]
    
    if 1<k<len(data): #Por seguridad no se crean siluetas con clusters menores a 2 y mayores a len(data)
        
        silhouette_avg = silhouette_score(data, labels)
        print("* Para el numero de clusters = ",k, " el promedio de las siluetas es de :",silhouette_avg)

        sample_silhouette_values = silhouette_samples(data, labels) #valores de cada silueta
          
        for i in range(k):
            ith_cluster_sv = sample_silhouette_values[labels == i]
            print("   - Para el Cluster ", i+1, " su silueta vale : ", np.mean(ith_cluster_sv))
            
            siluetas_temp.append(np.mean(ith_cluster_sv))
            
    resultado[k]=abs((np.max(siluetas_temp)-np.min(siluetas_temp)))

* Para el numero de clusters =  2  el promedio de las siluetas es de : 0.23424795518896774
   - Para el Cluster  1  su silueta vale :  0.28195856512188733
   - Para el Cluster  2  su silueta vale :  0.16067197484726334
* Para el numero de clusters =  3  el promedio de las siluetas es de : 0.20490325173788893
   - Para el Cluster  1  su silueta vale :  0.1515713311575713
   - Para el Cluster  2  su silueta vale :  0.16031581895305436
   - Para el Cluster  3  su silueta vale :  0.25388152068735875
* Para el numero de clusters =  4  el promedio de las siluetas es de : 0.18824813217190506
   - Para el Cluster  1  su silueta vale :  0.26241261596372356
   - Para el Cluster  2  su silueta vale :  0.1909584920880249
   - Para el Cluster  3  su silueta vale :  0.10368877335258073
   - Para el Cluster  4  su silueta vale :  0.13816054970113403
* Para el numero de clusters =  5  el promedio de las siluetas es de : 0.18096430372266253
   - Para el Cluster  1  su silueta vale :  0.1369794028419200

* Para el numero de clusters =  16  el promedio de las siluetas es de : 0.13111180784362875
   - Para el Cluster  1  su silueta vale :  0.22627373152478458
   - Para el Cluster  2  su silueta vale :  0.18465255462108823
   - Para el Cluster  3  su silueta vale :  0.07088614533841463
   - Para el Cluster  4  su silueta vale :  0.11229590336937471
   - Para el Cluster  5  su silueta vale :  0.1219224710205631
   - Para el Cluster  6  su silueta vale :  0.038193793609338667
   - Para el Cluster  7  su silueta vale :  0.16097749966268027
   - Para el Cluster  8  su silueta vale :  0.09518328341319011
   - Para el Cluster  9  su silueta vale :  0.10484229408459186
   - Para el Cluster  10  su silueta vale :  0.15214937870121778
   - Para el Cluster  11  su silueta vale :  0.247901219553715
   - Para el Cluster  12  su silueta vale :  0.104349840889793
   - Para el Cluster  13  su silueta vale :  0.10963066667160686
   - Para el Cluster  14  su silueta vale :  0.14710898816331547
   - Para e

* Para el numero de clusters =  23  el promedio de las siluetas es de : 0.13078198045115116
   - Para el Cluster  1  su silueta vale :  0.16029320392236632
   - Para el Cluster  2  su silueta vale :  0.13367479433410703
   - Para el Cluster  3  su silueta vale :  0.06523096642517857
   - Para el Cluster  4  su silueta vale :  0.17095320142268866
   - Para el Cluster  5  su silueta vale :  0.2029698828751121
   - Para el Cluster  6  su silueta vale :  0.09379208693235011
   - Para el Cluster  7  su silueta vale :  0.12391684320268707
   - Para el Cluster  8  su silueta vale :  0.06597739105245656
   - Para el Cluster  9  su silueta vale :  0.08843338772338392
   - Para el Cluster  10  su silueta vale :  0.23839354703694832
   - Para el Cluster  11  su silueta vale :  0.1298083372460388
   - Para el Cluster  12  su silueta vale :  0.04201636336080011
   - Para el Cluster  13  su silueta vale :  0.12613707093820314
   - Para el Cluster  14  su silueta vale :  0.11950483307935707
   - Para

* Para el numero de clusters =  28  el promedio de las siluetas es de : 0.1247248276384246
   - Para el Cluster  1  su silueta vale :  0.14645733672452313
   - Para el Cluster  2  su silueta vale :  0.1241110665351162
   - Para el Cluster  3  su silueta vale :  0.11826453649975277
   - Para el Cluster  4  su silueta vale :  0.14764405793560828
   - Para el Cluster  5  su silueta vale :  0.11016161520722338
   - Para el Cluster  6  su silueta vale :  0.1560615785661463
   - Para el Cluster  7  su silueta vale :  0.17001941139305302
   - Para el Cluster  8  su silueta vale :  0.0797681796367139
   - Para el Cluster  9  su silueta vale :  0.35873294624349783
   - Para el Cluster  10  su silueta vale :  0.04560021663139111
   - Para el Cluster  11  su silueta vale :  0.12664814741958064
   - Para el Cluster  12  su silueta vale :  0.08600334404842164
   - Para el Cluster  13  su silueta vale :  0.17435003649835654
   - Para el Cluster  14  su silueta vale :  0.11656247110493145
   - Para e

* Para el numero de clusters =  33  el promedio de las siluetas es de : 0.1346887020471128
   - Para el Cluster  1  su silueta vale :  0.09359828625179142
   - Para el Cluster  2  su silueta vale :  0.09910818477565017
   - Para el Cluster  3  su silueta vale :  0.13469195296614828
   - Para el Cluster  4  su silueta vale :  0.1502984578218111
   - Para el Cluster  5  su silueta vale :  0.15942319677690575
   - Para el Cluster  6  su silueta vale :  0.14784158368655514
   - Para el Cluster  7  su silueta vale :  0.1357823183458487
   - Para el Cluster  8  su silueta vale :  0.35038538810542125
   - Para el Cluster  9  su silueta vale :  0.12707023549558102
   - Para el Cluster  10  su silueta vale :  0.08399081448573965
   - Para el Cluster  11  su silueta vale :  0.13282628695164975
   - Para el Cluster  12  su silueta vale :  0.05949001169626477
   - Para el Cluster  13  su silueta vale :  0.10437099222292819
   - Para el Cluster  14  su silueta vale :  0.1349583373401622
   - Para e

...

* Para el numero de clusters =  183  el promedio de las siluetas es de : 0.18202906495748677
   - Para el Cluster  1  su silueta vale :  0.2042042004275971
   - Para el Cluster  2  su silueta vale :  0.05694185867118449
   - Para el Cluster  3  su silueta vale :  -0.014113466468168533
   - Para el Cluster  4  su silueta vale :  0.03611659857435742
   - Para el Cluster  5  su silueta vale :  0.20735614786933643
   - Para el Cluster  6  su silueta vale :  0.3483626217167168
   - Para el Cluster  7  su silueta vale :  0.15789790515687085
   - Para el Cluster  8  su silueta vale :  0.1106628574028061
   - Para el Cluster  9  su silueta vale :  0.26936327066705784
   - Para el Cluster  10  su silueta vale :  0.11739108783455802
   - Para el Cluster  11  su silueta vale :  0.07476731662431996
   - Para el Cluster  12  su silueta vale :  0.17024463800381165
   - Para el Cluster  13  su silueta vale :  0.2923465780026037
   - Para el Cluster  14  su silueta vale :  0.030376170266880265
   - Pa

* Para el numero de clusters =  184  el promedio de las siluetas es de : 0.18140675288222716
   - Para el Cluster  1  su silueta vale :  0.07745627400106056
   - Para el Cluster  2  su silueta vale :  0.21141816363758714
   - Para el Cluster  3  su silueta vale :  0.10562223626812556
   - Para el Cluster  4  su silueta vale :  0.3140169810024651
   - Para el Cluster  5  su silueta vale :  0.1527627518874902
   - Para el Cluster  6  su silueta vale :  0.15563601892122408
   - Para el Cluster  7  su silueta vale :  0.09536802369285963
   - Para el Cluster  8  su silueta vale :  0.21830504519131677
   - Para el Cluster  9  su silueta vale :  0.03541443860809181
   - Para el Cluster  10  su silueta vale :  0.20717494146080875
   - Para el Cluster  11  su silueta vale :  0.007697992492232135
   - Para el Cluster  12  su silueta vale :  0.7213004543970032
   - Para el Cluster  13  su silueta vale :  0.21720491948399176
   - Para el Cluster  14  su silueta vale :  0.2833558054848064
   - Para

* Para el numero de clusters =  185  el promedio de las siluetas es de : 0.18506123378474126
   - Para el Cluster  1  su silueta vale :  0.17437400685509913
   - Para el Cluster  2  su silueta vale :  0.040814489445178154
   - Para el Cluster  3  su silueta vale :  0.1243125901460067
   - Para el Cluster  4  su silueta vale :  0.029815657779214303
   - Para el Cluster  5  su silueta vale :  0.10091139345409736
   - Para el Cluster  6  su silueta vale :  0.21950862696307577
   - Para el Cluster  7  su silueta vale :  0.3408391597377036
   - Para el Cluster  8  su silueta vale :  0.06544330047053667
   - Para el Cluster  9  su silueta vale :  0.8575661177254745
   - Para el Cluster  10  su silueta vale :  0.1923637173338876
   - Para el Cluster  11  su silueta vale :  0.2463639922806306
   - Para el Cluster  12  su silueta vale :  0.25688050607727697
   - Para el Cluster  13  su silueta vale :  0.10693127050399086
   - Para el Cluster  14  su silueta vale :  0.19958579086243625
   - Para

* Para el numero de clusters =  186  el promedio de las siluetas es de : 0.18497003070267803
   - Para el Cluster  1  su silueta vale :  0.20035525751667263
   - Para el Cluster  2  su silueta vale :  0.4147430068999267
   - Para el Cluster  3  su silueta vale :  0.10315622102414042
   - Para el Cluster  4  su silueta vale :  0.19169531756532432
   - Para el Cluster  5  su silueta vale :  0.1819930611815931
   - Para el Cluster  6  su silueta vale :  0.13676518961593195
   - Para el Cluster  7  su silueta vale :  0.15026228729610142
   - Para el Cluster  8  su silueta vale :  0.32132865144661366
   - Para el Cluster  9  su silueta vale :  0.12002900962454119
   - Para el Cluster  10  su silueta vale :  0.18035123628516347
   - Para el Cluster  11  su silueta vale :  0.0
   - Para el Cluster  12  su silueta vale :  0.12663117407073093
   - Para el Cluster  13  su silueta vale :  0.1975519641528723
   - Para el Cluster  14  su silueta vale :  0.27014760800766807
   - Para el Cluster  15 

KeyboardInterrupt: 

In [80]:
print(len(resultado))
print('El numero de clusters optimo es: '+str(min(resultado, key=resultado.get)))


185
El numero de clusters optimo es: 3


### Metodo del Codo

In [81]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

In [82]:
max_clusters=len(data)
clusters=range(2,max_clusters) #clusters a partir de 2

In [83]:
max_clust=len(data)
clusters=range(2,max_clust)#importante crear mas de 2 clusters para el metodo de silueta y codo
ssw=[] #suma de los cuadrados internos

for k in clusters:
    kmean_model=KMeans(n_clusters=k).fit(data)
    
    ssw_k=sum(np.min(cdist(data,kmean_model.cluster_centers_,'euclidean'),axis=1))
    ssw.append(ssw_k)
    print('Test con numero de clusters: '+str(k))    

Test con numero de clusters: 2
Test con numero de clusters: 3
Test con numero de clusters: 4
Test con numero de clusters: 5
Test con numero de clusters: 6
Test con numero de clusters: 7
Test con numero de clusters: 8
Test con numero de clusters: 9
Test con numero de clusters: 10
Test con numero de clusters: 11
Test con numero de clusters: 12
Test con numero de clusters: 13
Test con numero de clusters: 14
Test con numero de clusters: 15
Test con numero de clusters: 16
Test con numero de clusters: 17
Test con numero de clusters: 18
Test con numero de clusters: 19
Test con numero de clusters: 20
Test con numero de clusters: 21
Test con numero de clusters: 22
Test con numero de clusters: 23
Test con numero de clusters: 24
Test con numero de clusters: 25
Test con numero de clusters: 26
Test con numero de clusters: 27
Test con numero de clusters: 28
Test con numero de clusters: 29
Test con numero de clusters: 30
Test con numero de clusters: 31
Test con numero de clusters: 32
Test con numero 

Test con numero de clusters: 254
Test con numero de clusters: 255
Test con numero de clusters: 256
Test con numero de clusters: 257
Test con numero de clusters: 258
Test con numero de clusters: 259
Test con numero de clusters: 260
Test con numero de clusters: 261
Test con numero de clusters: 262
Test con numero de clusters: 263
Test con numero de clusters: 264
Test con numero de clusters: 265
Test con numero de clusters: 266
Test con numero de clusters: 267
Test con numero de clusters: 268
Test con numero de clusters: 269
Test con numero de clusters: 270
Test con numero de clusters: 271
Test con numero de clusters: 272
Test con numero de clusters: 273
Test con numero de clusters: 274
Test con numero de clusters: 275
Test con numero de clusters: 276
Test con numero de clusters: 277
Test con numero de clusters: 278
Test con numero de clusters: 279
Test con numero de clusters: 280
Test con numero de clusters: 281
Test con numero de clusters: 282
Test con numero de clusters: 283
Test con n

Test con numero de clusters: 503
Test con numero de clusters: 504
Test con numero de clusters: 505


KeyboardInterrupt: 

In [84]:
ssw2=np.diff(ssw) 
ssw2=[abs(num) for num in ssw2]

max=0

for index_i,i in enumerate(ssw2):
    if(i>max):
        max=i
        idx=index_i
    for index_j,j in enumerate(ssw2):
        if(j>max):
            max=j
            idx=index_j          
print('Numero optimo de clusters: '+str(clusters[idx+1]))

Numero optimo de clusters: 3
